# ChatData

## Sqlite
We are using a file system called **sqlite**. It looks and acts like a real single user relational database (RDB). sqlite3 comes packaged with python. 

- We are using Pandas to write and read to sqlite. Pandas will manage a lot of the complexity of dealing with a RDB. 

- This activity is primarily about querying a RDB, so, we are using a simple way that Pandas provides. 

- All of the SQL queries could also be performed on the Pandas DataFrames directly.  

## SQL Magic
Within the Jupyter notebook we will be using **SQL Magic**.  This provides a convenient way to write SQL queries directly into code cells in the notebook and to read the results back into a Pandas DataFrame.  



# Imports
First let's import Sqlite and the other libraries we will need.

In [38]:
# Import Libraries
import numpy as np
import pandas as pd
import sqlite3

### Create the Database
Now we will create the Sqlite database. We use the `%load_ext` magic command to load the SQL Magic extension and then use `%sql` to connect to the database.

In [39]:
# If the db does not exist, sqlite will create it.
con = sqlite3.connect('chatdata.db')

# loads sql magic
%load_ext sql 

# connects sql magic command to the correct db
%sql sqlite:///chatdata.db 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @chatdata.db'

#### Drop the `queries` table if it already exists
The queries table will be our record of the queries created to answer the questions from the requirements spreadsheet.  As we will be running this Jupyter notebook a few times, let's drop (i.e. remove) the queries table so that we start fresh each time.  Here is the code to do this.  We use the `%%sql` magic command to tell Jupyter that we are going to write SQL in the cell.


In [40]:
%%sql
DROP TABLE IF EXISTS 'queries'

 * sqlite:///chatdata.db
Done.


[]

# 1: Load the Data
Now we can start loading the data.  The tables will be created as we load the data.  

## Lifecycle Stages: Transform and Organise
The data has already been processed into 3 clean data files ready for this project:

- queries.csv
- posts.csv
- comments.csv

We will load these files into our database.

## Lifecycle Stage: Acquire


### Load Comments Data into a `comments` table
Now we will load the data from the csv files into our sqlite database.

First we load the csv file into a Pandas dataframe:

In [41]:
comments = pd.read_csv('comments.csv')
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


Now take the comments dataframe and push the data into the Sqlite database table called 'comments':

In [42]:
# load comments into sqlite
comments.to_sql('comments', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM comments'
comments = pd.read_sql(sql, con)
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


### Load Other CSVs


In [43]:
posts = pd.read_csv('posts.csv')
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,NaN,...,NaN,NaN,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,NaN
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,NaN,...,NaN,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,NaN,NaN
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,NaN,...,NaN,NaN,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,NaN,NaN
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,NaN,...,NaN,NaN,2019-08-24 09:57:01,NaN,NaN,0,0,0,NaN,NaN
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,NaN,...,NaN,2019-08-24 12:13:46,2019-08-24 12:13:46,NaN,NaN,0,1,0,NaN,NaN


In [44]:
# load posts into sqlite
posts.to_sql('posts', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM posts'
posts = pd.read_sql(sql, con)
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


In [45]:
users = pd.read_csv('users.csv')
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,NaN,NaN,NaN,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,NaN,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,NaN,NaN,NaN,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,NaN,NaN,NaN,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [46]:
# load users into sqlite
users.to_sql('users', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM users'
users = pd.read_sql(sql, con)
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


### Drop Duplicates


In [47]:
# Find out if we have any duplicates
users["Id"].value_counts()

157607    1
35998     1
182632    1
182594    1
182590    1
         ..
243754    1
243759    1
243763    1
243765    1
157592    1
Name: Id, Length: 18412, dtype: int64

In [48]:
# Find out if we have any duplicates
posts["Id"].value_counts()

423497    1
420405    1
420409    1
420410    1
420412    1
         ..
402692    1
402693    1
402694    1
402695    1
438743    1
Name: Id, Length: 42234, dtype: int64

In [49]:
# Find out if we have any duplicates
comments["Id"].value_counts()

723182    1
769788    1
768879    1
768880    1
768881    1
         ..
746468    1
746469    1
746470    1
746472    1
791753    1
Name: Id, Length: 50000, dtype: int64

## Review the Data

In [50]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [51]:
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [52]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [53]:
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


In [54]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [171]:
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


In Sqlite we need to enable the enforcement of foreign key constraints:

In [62]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///chatdata.db
Done.


[]

Now create the users table with a **primary key constraint** 

In [63]:
%%sql
CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);


 * sqlite:///chatdata.db
Done.


[]

In [64]:
%%sql
CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);

 * sqlite:///chatdata.db
Done.


[]

In [67]:
%%sql
CREATE TABLE "comments" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostId" INTEGER,
"Score" INTEGER,
"Text" TEXT,
"CreationDate" TEXT,
"UserId" INTEGER,
FOREIGN KEY(PostId) REFERENCES posts(Id),
FOREIGN KEY(UserId) REFERENCES users(Id)
)

 * sqlite:///chatdata.db
Done.


[]

Now we can insert the data into these constrained tables.  First users:

In [68]:
# Insert data into the new users table
users.to_sql('users', con, if_exists='append', index=False)

18412

In [69]:
# Insert data into the new posts table
posts.to_sql('posts', con, if_exists='append', index=False)

42234

In [70]:
# Insert data into the new comments table
comments.to_sql('comments', con, if_exists='append', index=False)

50000

Now check that we have the 3 new table definitions in Sqlite:

In [71]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name
comments
posts
users


We now have all the data in tables in Sqlite and the tables will enforce the referential integrity.

1. Prepare the Sqlite query to answer the question
2. Test it
3. Insert it into the `queries` table, so we have a record of it for others.

This is the process that we want you to follow for this project while completing it. 

Let's see an example of this by answering the following question:

**Which 5 users have viewed the most times and what is the sum of those views per user?**

## Prepare the Sqlite query

First, let's write the query:

In [325]:
sql = """
SELECT Id, SUM(Views) AS TotalViews
    FROM Users
        GROUP BY Id
            ORDER BY TotalViews DESC
                LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
result

,Id,TotalViews
0,919,85180
1,4253,35119
2,805,34637
3,7290,32639
4,3277,29255


## Test the query
You can optionally prove the query worked by performing the same query in Pandas:

In [73]:
results = users.groupby(['Id']).sum().sort_values('Views', ascending = False)[:5]
results['Views']

Id
919     85180
4253    35119
805     34637
7290    32639
3277    29255
Name: Views, dtype: int64

## Insert the query into the `queries` table
Now we need to put this query into the `queries` table in sqlite. Remember we want these queries to be accessible to everybody that should have access to them. We do not want people writing and rewriting the same queires over and over again. The easiest thing to do is create a dictionary with the values and insert these into the queries table.  Note that the values are provided as lists as we are inserting a list of values (i.e. a number of rows) into the table.  In this case the number of rows is 1, so we have lists of 1 item.

So here, we have a column called 'task' with a list of values, a column called 'action' with a list of values, etc.

In [631]:
query_dict = {
              'task': ['Single Table Queries'],
              'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
              'query': [sql]
             }
query_dict

{'task': ['Single Table Queries'],
 'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
 'query': ['SELECT * FROM queries']}

Now that you have the data structure (query_dict) containing the data, create a pandas dataframe that holds those values:

In [632]:
queries = pd.DataFrame(query_dict)
queries

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,SELECT * FROM queries


Now load that pandas dataframe (queries) into the sqlite table called queries. In this case, you use append NOT replace. You will be adding to this tables as you go thru this project.

In [633]:
# load query into sqlite
queries.to_sql('queries', con, if_exists='append', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)
queries.head()

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,SELECT * FROM queries


# Insert the CREATE TABLE Statements into the `queries` Table




In [634]:
# Define a function that will insert into the queries table
def store_query(task, action, query):
    query_dict = {
                  'task': [task],
                  'action': [action],
                  'query': [query]
                 }

    # put query into the query_dict
    queries = pd.DataFrame(query_dict)

    # load query into sqlite
    queries.to_sql('queries', con, if_exists='append', index=False)    

Now we can specify the queries and call the above function to store them. 

In [635]:
sql = """
    CREATE TABLE "comments" (
    "Id" INTEGER,
    "PostId" INTEGER,
    "Score" INTEGER,
    "Text" TEXT,
    "CreationDate" TEXT,
    "UserId" INTEGER
    )
    """
store_query("Task 1", "Create table comments", sql)

Let's prove it works by selecting back from the queries table:

In [636]:
# Prove it works
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"


In [637]:
# Insert the CREATE TABLE for posts into the queries table
sql = """
    CREATE TABLE "posts" (
    "Id" INTEGER,
    "PostTypeId" INTEGER,
    "AcceptedAnswerId" INTEGER,
    "ParentId" INTEGER,
    "CreationDate" TEXT,
    "Score" INTEGER,
    "ViewCount" INTEGER,
    "Body" TEXT,
    "OwnerUserId" INTEGER,
    "OwnerDisplayName" TEXT,
    "LastEditorUserId" INTEGER,
    "LastEditorDisplayName" TEXT,
    "LastEditDate" TEXT,
    "LastActivityDate" TEXT,
    "Title" TEXT,
    "Tags" TEXT,
    "AnswerCount" INTEGER,
    "CommentCount" INTEGER,
    "FavoriteCount" INTEGER,
    "ClosedDate" TEXT,
    "CommunityOwnedDate" TEXT
    )
    """
store_query("Task 2", "Create table post", sql)

In [638]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"


In [639]:
# Insert the CREATE TABLE for users into the queries table
sql = """
    CREATE TABLE "users" (
    "Id" INTEGER,
    "Reputation" INTEGER,
    "CreationDate" TEXT,
    "DisplayName" TEXT,
    "LastAccessDate" TEXT,
    "WebsiteUrl" TEXT,
    "Location" TEXT,
    "AboutMe" TEXT,
    "Views" INTEGER,
    "UpVotes" INTEGER,
    "DownVotes" INTEGER,
    "ProfileImageUrl" TEXT,
    "AccountId" INTEGER
    )
    """
store_query("Task 3", "Create table users", sql)

In [640]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"


## Count the Number of Rows in Each Table


In [641]:
# Count the number of rows in the comments table
%sql SELECT Count(*) AS NumberOfRows FROM comments


 * sqlite:///chatdata.db
Done.


NumberOfRows
50000


In [642]:
# insert the query into the queries table
sql = """
SELECT Count(*) AS NumberOfRows
    FROM comments
        """
store_query("Task 4", "Count the number of rows in the comments table", sql)

In [643]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments


In [644]:
# Count the number of rows in the posts table
%sql SELECT Count(*) AS NumberOfRows FROM posts

 * sqlite:///chatdata.db
Done.


NumberOfRows
42234


In [645]:
# insert the query into the queries table
sql = """
SELECT Count(*) AS NumberOfRows
    FROM posts
        """
store_query("Task 5", "Count the number of rows in the posts table", sql)

In [646]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments
Task 5,Count the number of rows in the posts table,SELECT Count(*) AS NumberOfRows FROM posts


In [647]:
# Count the number of rows in the users table
%sql SELECT Count(*) AS NumberOfRows FROM users

 * sqlite:///chatdata.db
Done.


NumberOfRows
18412


In [648]:
# insert the query into the queries table
sql = """
SELECT Count(*) AS NumberOfRows
    FROM users
        """
store_query("Task 6", "Count the number of rows in the users table", sql)

In [649]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments
Task 5,Count the number of rows in the posts table,SELECT Count(*) AS NumberOfRows FROM posts
Task 6,Count the number of rows in the users table,SELECT Count(*) AS NumberOfRows FROM users


## Do some Random Checks on the Data
Let's write some queries that select 5 random rows from each table.  The queries are provided here:

    select * from Comments order by random() limit 5;
    select * from Users order by random() limit 5;
    select * from Posts order by random() limit 5;



In [650]:
# Run the query to select 5 random rows from the posts table
%sql SELECT * FROM posts ORDER BY random() LIMIT 5;

 * sqlite:///chatdata.db
Done.


Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
415727,1,0,0,2019-07-02 16:34:54,6,125,"<p>In <a href=""https://wmbriggs.com/post/18995/"" rel=""noreferrer"">this blog</a> by William Briggs, who seems to be a prolific lecturer/writer on pop-statistics, he condemns the ""Bayesian Metaphor"" which is essentially referring to Bayesian probability as belief. </p><blockquote> <p>Quoting from a post on vampires, “In Bayesian inference, you start with some initial beliefs (called ‘Bayesian priors’ or just ‘priors’), and then you ‘update’ them as you receive new evidence.”</p> <p>This is the standard metaphor, and it’s not so much wrong as unhelpful, misleading, and restricting. The metaphor derives from Bayes’s rule (details which can be looked up anywhere) and which gives a formula which on the right-hand-side is supposed to be an element representing “prior beliefs.” The formula itself is correct, as most math is. But because math is correct does not mean that it means what you think it means.</p></blockquote><p>In my opinion, the prior being a ""belief"" is not a metaphor, but it is a veritable reflection of Bayesian probability. For instance, Bayesians can probabilistically quantify events that frequentists cannot such as the location of a quantum particle. I understand the distinction between Bayes' Rule and Bayesian Statistics: the former is a probabilistic law that applies to frequentist as well as Bayesian inference as well; the latter deals with a Bayesian interpretation of probability. He goes on to say:</p><blockquote> <p>This is wrong because there is no such thing as “Pr(Y)” or “Pr(X)”. These objects do not exist. Numbers can be put in their place and the equation can be made to work out, but it is the step of putting numbers in that is wrong. There is no such thing as an unconditional probability, so we can never write without error “Pr(Y)” or “Pr(X)”. Instead, we should write e.g. Pr(Y|W) or Pr(X|W), where W is the knowledge we start with, i.e. our real prior (knowledge).</p></blockquote><p>And I think at this point--that this post doesn't seem to have anything to do with Bayesian statistics at all. Like a frequentist putting uncertainty bounds on uncertainty bounds, it seems the Bayesian analogue is conditioning on conditioning. </p><p>Is this blog sound reasoning? Is there actually a distinction between ""belief"" and ""knowledge"" when specifying a prior? If a conditional density exists, such as <span class=""math-container"">$Pr(Y|W)$</span> then a marginal density exists as well <span class=""math-container"">$Pr(Y) = \int Pr(Y|W)Pr(W)$</span>?</p>",8013,None,0,None,None,2019-07-03 00:39:18,"If the Bayesian probability is not a belief, what is it?",<probability><bayesian>,3,3,3,None,None,2554
393742,2,0,393721,2019-02-21 21:24:03,1,0,"<p>The advantage of a Cox PH model is that it is designed to handle censoring, so long as the censoring is <a href=""https://stats.stackexchange.com/q/65202/28500"">non-informative</a>.* For example, the presence of censoring should contain no information about the actual time to event except that the event occurred later than the censoring time. Beyond that requirement there is no ""acceptable limit"" on censoring. For example, in a disease with low recurrence or mortality there will necessarily be a large fraction of censored cases in analysis of disease-specific survival. The power of your analysis will essentially be set by the number of events; with 200 events you should be able to evaluate a reasonably complex survival model with a dozen or so predictors.</p><p>What might be useful to allay your fears and those of your reviewers or readers would be to display the distributions of censoring and event times in a way that they can be compared, say as superimposed separate

In [651]:
# insert the query into the queries table
sql = """
    SELECT * 
    FROM posts 
    ORDER BY random() 
    LIMIT 5
        """
store_query("Task 7", "select 5 random rows from posts table", sql)

In [652]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments
Task 5,Count the number of rows in the posts table,SELECT Count(*) AS NumberOfRows FROM posts
Task 6,Count the number of rows in the users table,SELECT Count(*) AS NumberOfRows FROM users
Task 7,select 5 random rows from posts table,SELECT * FROM posts ORDER BY random() LIMIT 5


In [653]:
# Run the query to select 5 random rows from the posts comments
%sql SELECT * FROM comments ORDER BY random() LIMIT 5;

 * sqlite:///chatdata.db
Done.


Id,PostId,Score,Text,CreationDate,UserId
753672,402245,0,"Thank you for your further comment. In my case-control data, case and control samples were collected from different cohorts. The case samples were collected from a hospital-based cohort. The control samples were collected from a population-based cohort. In that case, is it appropriate that a correlation analysis is performed using the case-control study based data?",2019-04-13 04:59:12,161341
766029,410120,3,"@naveganteX, example: 100 samples, 50-25-25 train/validation/test split. With CV, the first 75 of that are used for training+validation and every sample of that 75 is used for both training and validation (that's what CV is). The remaining 25 of the test data is there no matter what.",2019-05-26 06:17:10,241093
784321,420394,0,"With $\nu$ = N-1 = sample size - 1, try overplotting with $\Gamma(\nu/2 , 2\sigma^2 /\nu)$. So for N = 7, use $\Gamma(3, 8.3333)$ and for N = 30, use $\Gamma(14.5, 1.72414)$. There is a nice answer at this site that shows how the gamma distribution is related to the chi square distribution. As promised: https://stats.stackexchange.com/q/118676/247352",2019-08-03 00:17:35,247352
768152,411368,0,"You don't need to fit a function in order simulate from the distribution--as I suggested earlier, fitting the function has its uses, but simulation is not one of the better ones when you have so much data.",2019-06-03 19:26:51,919
768006,129849,1,Please keep in mind that the package seems to rely on factor orderings. The package does not seem to give sensible results for unordered variables.,2019-06-03 11:44:34,106454


In [654]:
# insert the query into the queries table
sql = """
    SELECT * 
    FROM comments 
    ORDER BY random() 
    LIMIT 5
        """
store_query("Task 8", "select 5 random rows from comments table", sql)

In [107]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments
Task 5,Count the number of rows in the posts table,SELECT Count(*) AS NumberOfRows FROM posts
Task 6,Count the number of rows in the users table,SELECT Count(*) AS NumberOfRows FROM users
Task 7,select 5 random rows from posts table,SELECT * FROM posts ORDER BY random() LIMIT 5
Task 8,select 5 random rows from comments table,SELECT * FROM comments ORDER BY random() LIMIT 5


In [655]:
# Run the query to select 5 random rows from the users table
%sql SELECT * FROM users ORDER BY random() LIMIT 5;

 * sqlite:///chatdata.db
Done.


Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
250992,23,2019-06-15 05:13:50,user677216,2019-08-11 23:59:11,None,None,None,2,0,0,https://www.gravatar.com/avatar/eb5d1368672795e12587ad9a65b5865f?s=128&d=identicon&r=PG&f=1,16012774
124277,33,2016-07-23 20:35:26,R_user123,2019-09-07 01:47:18,None,None,None,12,6,0,https://www.gravatar.com/avatar/b3fb69e171f312d8cde4f1245fe15645?s=128&d=identicon&r=PG&f=1,7062891
254291,303,2019-07-21 13:32:44,davo.biainili,2019-11-15 04:12:43,None,"Boston, MA, USA","<p>One humid, brown dusk -The rain is emanating:Smell of the asphalt.</p><p>Just an enthusiast.</p>",18,10,0,https://i.stack.imgur.com/EegOH.jpg,14743720
41561,390,2014-03-08 20:06:32,Kam Sen,2019-11-26 12:37:53,None,None,None,19,1,0,https://www.gravatar.com/avatar/fb1933c8a098b769968a3d63f021709e?s=128&d=identicon&r=PG,2343614
252101,53,2019-06-27 09:36:38,amars96,2019-08-27 12:39:29,None,None,None,8,1,0,https://www.gravatar.com/avatar/9ba13abbb738e4ae672620eec8ddd6a6?s=128&d=identicon&r=PG&f=1,16214615


In [656]:
# insert the query into the queries table
sql = """
    SELECT * 
    FROM users 
    ORDER BY random() 
    LIMIT 5
        """
store_query("Task 9", "select 5 random rows from users table", sql)

In [657]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments
Task 5,Count the number of rows in the posts table,SELECT Count(*) AS NumberOfRows FROM posts
Task 6,Count the number of rows in the users table,SELECT Count(*) AS NumberOfRows FROM users
Task 7,select 5 random rows from posts table,SELECT * FROM posts ORDER BY random() LIMIT 5
Task 8,select 5 random rows from comments table,SELECT * FROM comments ORDER BY random() LIMIT 5
Task 9,select 5 random rows from users table,SELECT * FROM users ORDER BY random() LIMIT 5


# Create Single Table Queries


## Lifecycle Stage: Analyze
We can now start the analysis with our single-table queries.  First we need to create a new computed column to help with one of the queries.  The code below creates a column called LEN_BODY which is the length of the BODY text:


In [658]:
%%sql
ALTER TABLE POSTS ADD COLUMN LEN_BODY INT

 * sqlite:///chatdata.db
(sqlite3.OperationalError) duplicate column name: LEN_BODY
[SQL: ALTER TABLE POSTS ADD COLUMN LEN_BODY INT]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [111]:
%%sql
UPDATE POSTS SET LEN_BODY = LENGTH(BODY)

 * sqlite:///chatdata.db
42234 rows affected.


[]

### Single Table Queries


<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many posts have 0 comments?
    </font></div>

In [666]:
%%sql
SELECT COUNT(*) "Number of Comments"
FROM posts
WHERE CommentCount = 0

 * sqlite:///chatdata.db
Done.


Number of Comments
21713


In [667]:
# insert the query into the queries table
sql = """
    SELECT COUNT(*) "Number of Comments"
    FROM posts
    WHERE CommentCount = 0
        """
store_query("Task 10", "How many posts have 0 comments?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many posts have 1 comments?
    </font></div>

In [668]:
%%sql
SELECT COUNT(*) "Number of Comments"
FROM posts
WHERE CommentCount = 1

 * sqlite:///chatdata.db
Done.


Number of Comments
6460


In [669]:
# insert the query into the queries table
sql = """
    SELECT COUNT(*) "Number of Comments"
    FROM posts
    WHERE CommentCount = 1
        """
store_query("Task 11", "How many posts have 1 comments?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many posts have 2 comments or more?
    </font></div>

In [670]:
%%sql
SELECT COUNT(*) "Number of Comments"
FROM posts
WHERE CommentCount >= 2

 * sqlite:///chatdata.db
Done.


Number of Comments
14061


In [671]:
# insert the query into the queries table
sql = """
    SELECT COUNT(*) "Number of Comments"
    FROM posts
    WHERE CommentCount >= 2
        """
store_query("Task 12", "How many posts have 2 comments or more?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Find the 5 posts with the highest viewcount
    </font></div>

In [672]:
%%sql 
SELECT Id "Post Id", SUM(ViewCount) "Total Views"
FROM posts
GROUP BY Id
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///chatdata.db
Done.


Post Id,Total Views
388566,19542
394118,16317
431370,11723
398646,9850
434128,6718


In [673]:
# insert the query into the queries table
sql = """
    SELECT Id "Post Id", SUM(ViewCount) "Total Views"
    FROM posts
    GROUP BY Id
    ORDER BY 2 DESC
    LIMIT 5
        """
store_query("Task 13", "Find the 5 posts with the highest viewcount", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Find the top 5 posts with the highest scores
    </font></div>

In [674]:
%%sql 
SELECT Id "Post Id", SUM(Score) "Total Score"
FROM posts
GROUP BY Id
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///chatdata.db
Done.


Post Id,Total Score
431397,101
394128,100
426878,93
388578,80
431370,77


In [675]:
# insert the query into the queries table
sql = """
    SELECT Id "Post Id", SUM(Score) "Total Score"
    FROM posts
    GROUP BY Id
    ORDER BY 2 DESC
    LIMIT 5
        """
store_query("Task 14", "Find the top 5 posts with the highest scores", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What are the 5 most frequent scores on posts? 
    </font></div>

In [677]:
%%sql 
SELECT Score, COUNT(Score) "Frequent Scores on Posts"
FROM posts
GROUP BY Score
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///chatdata.db
Done.


Score,Frequent Scores on Posts
0,19888
1,11867
2,5094
3,2228
4,1059


In [678]:
# insert the query into the queries table
sql = """
    SELECT Score, COUNT(Score) "Frequent Scores on Posts"
    FROM posts
    GROUP BY Score
    ORDER BY 2 DESC
    LIMIT 5
        """
store_query("Task 15", "What are the 5 most frequent scores on posts?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many posts have the keyword "data" in their tags?
    </font></div>

In [679]:
%%sql
SELECT Count(*) "Number of Posts"
FROM posts
WHERE Tags LIKE '%data%' 

 * sqlite:///chatdata.db
Done.


Number of Posts
2242


In [680]:
# insert the query into the queries table
sql = """
    SELECT Count(*) "Number of Posts"
    FROM posts
    WHERE Tags LIKE '%data%' 
        """
store_query("Task 16", "How many posts have the keyword data in their tags?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What are the 5 most frequent commentcount for posts? REVIEW
    </font></div>

In [681]:
%%sql 
SELECT CommentCount, COUNT(CommentCount) "Frequent Comment Count"
FROM posts
GROUP BY CommentCount
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///chatdata.db
Done.


CommentCount,Frequent Comment Count
0,21713
1,6460
2,4966
3,3063
4,2026


In [682]:
# insert the query into the queries table
sql = """
    SELECT CommentCount, COUNT(CommentCount) "Frequent Comment Count"
    FROM posts
    GROUP BY CommentCount
    ORDER BY 2 DESC
    LIMIT 5
        """
store_query("Task 17", "What are the 5 most frequent commentcount for posts?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many posts have an accepted answer?
    </font></div>

In [683]:
%%sql 
SELECT COUNT(*) "Number of accepted answer"
FROM posts
WHERE AcceptedAnswerId <> 0 

 * sqlite:///chatdata.db
Done.


Number of accepted answer
5341


In [684]:
# insert the query into the queries table
sql = """
    SELECT COUNT(*) "Number of accepted answer"
    FROM posts
    WHERE AcceptedAnswerId <> 0
        """
store_query("Task 18", "How many posts have an accepted answer?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What is the average reputation of table users?
    </font></div>

In [685]:
%%sql
SELECT ROUND(AVG(Reputation),2) "User Reputation AVG"
FROM users

 * sqlite:///chatdata.db
Done.


User Reputation AVG
312.35


In [686]:
# insert the query into the queries table
sql = """
    SELECT ROUND(AVG(Reputation),2) "User Reputation AVG"
    FROM users
        """
store_query("Task 19", "What is the average reputation of table users?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What are the min and max reputation of users?
    </font></div>

In [687]:
%%sql
SELECT MIN(Reputation) "Min Reputation", MAX(Reputation) "Max Reputation"
FROM users

 * sqlite:///chatdata.db
Done.


Min Reputation,Max Reputation
1,228662


In [688]:
# insert the query into the queries table
sql = """
    SELECT MIN(Reputation) "Min Reputation", MAX(Reputation) "Max Reputation"
    FROM users
        """
store_query("Task 20", "What are the min and max reputation of users?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What is the length of the body of 5 most viewed posts?
    </font></div>

In [689]:
%%sql 
SELECT Id "Post Id", SUM(ViewCount) "Total Views", LEN_BODY "Length"
FROM posts
GROUP BY Id
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///chatdata.db
Done.


Post Id,Total Views,Length
388566,19542,2270
394118,16317,512
431370,11723,811
398646,9850,2148
434128,6718,1172


In [690]:
# insert the query into the queries table
sql = """
    SELECT Id "Post Id", SUM(ViewCount) "Total Views", LEN_BODY "Length"
    FROM posts
    GROUP BY Id
    ORDER BY 2 DESC
    LIMIT 5
        """
store_query("Task 21", "What is the length of the body of 5 most viewed posts?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many different locations are there in the users table?
    </font></div>

In [691]:
%%sql
SELECT COUNT(DISTINCT(Location)) "Different Locations"
FROM users 

 * sqlite:///chatdata.db
Done.


Different Locations
1900


In [692]:
# insert the query into the queries table
sql = """
    SELECT COUNT(DISTINCT(Location)) "Different Locations"
    FROM users  
        """
store_query("Task 22", "How many different locations are there in the users table?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What are the top 5 locations of users?
    </font></div>

In [694]:
%%sql 
SELECT Location, COUNT(Location) "Users"
FROM users
GROUP BY Location
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///chatdata.db
Done.


Location,Users
Germany,117
India,100
United States,69
"Paris, France",66
"London, United Kingdom",63


In [695]:
# insert the query into the queries table
sql = """
    SELECT Location, COUNT(Location) "Users"
    FROM users
    GROUP BY Location
    ORDER BY 2 DESC
    LIMIT 5
        """
store_query("Task 23", "What are the top 5 locations of users?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Rank the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage. 
    </font></div>

In [700]:
%%sql
  select case cast (strftime('%w', CreationDate) as integer)
  when 0 then 'Sunday'
  when 1 then 'Monday'
  when 2 then 'Tuesday'
  when 3 then 'Wednesday'
  when 4 then 'Thursday'
  when 5 then 'Friday'
  else 'Saturday' end as Day, ROUND(SUM(ViewCount)*100.0/(select SUM(ViewCount) from posts),2) as SumViewCountPercent
    FROM posts  
    GROUP BY Day
    ORDER BY 2 DESC

 * sqlite:///chatdata.db
Done.


Day,SumViewCountPercent
Thursday,16.82
Wednesday,16.77
Tuesday,16.26
Monday,15.81
Friday,13.56
Sunday,11.9
Saturday,8.89


In [701]:
# insert the query into the queries table
sql = """
  select case cast (strftime('%w', CreationDate) as integer)
  when 0 then 'Sunday'
  when 1 then 'Monday'
  when 2 then 'Tuesday'
  when 3 then 'Wednesday'
  when 4 then 'Thursday'
  when 5 then 'Friday'
  else 'Saturday' end as Day, ROUND(SUM(ViewCount)*100.0/(select SUM(ViewCount) from posts),2) as SumViewCountPercent
    FROM posts  
    GROUP BY Day
    ORDER BY 2 DESC
        """
store_query("Task 24", "Rank the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage.", sql)

# 3: Cross Table Queries

## Lifecycle Stage: Analyze
Let's continue the analysis with our multi-table queries.  

### Cross Table Queries





<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
How many posts have been created by a user that has a filled out the "AboutMe" section? 
    </font></div>

In [702]:
%%sql
SELECT COUNT(*) "Posts"
    FROM users u INNER JOIN posts p ON u.Id = p.OwnerUserId
    WHERE u.AboutMe <> '';

 * sqlite:///chatdata.db
Done.


Posts
17189


In [703]:
# insert the query into the queries table
sql = """
    SELECT COUNT(*) "Posts"
    FROM users u INNER JOIN posts p ON u.Id = p.OwnerUserId
    WHERE u.AboutMe <> '';
        """
store_query("Task 25", "How many posts have been created by a user that has a filled out the AboutMe section?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Considering only the users with an "AboutMe," how many posts are there per user? count all divide by subquery
    </font></div>

In [704]:
%%sql
WITH 
T1 AS (SELECT COUNT(*) AS Posts FROM users u INNER JOIN posts p ON u.Id = p.OwnerUserId WHERE u.AboutMe !=''),
T2 AS (SELECT CAST(COUNT(*) AS FLOAT) AS Users FROM users WHERE AboutMe !='') 

SELECT ROUND(CAST(T1.Posts/T2.Users AS FLOAT),2) "Post per Users"
FROM T1, T2

 * sqlite:///chatdata.db
Done.


Post per Users
4.27


In [699]:
%%sql
SELECT ROUND(COUNT(*)/(SELECT CAST(COUNT(*) AS FLOAT) FROM USERS WHERE ABOUTME !=''),2) AS posts_per_user
    FROM USERS U
    JOIN POSTS P ON U.ID = P.OWNERUSERID  
    WHERE U.ABOUTME !=''

 * sqlite:///chatdata.db
Done.


posts_per_user
4.27


In [705]:
# insert the query into the queries table
sql = """
    SELECT ROUND(COUNT(*)/(SELECT CAST(COUNT(*) AS FLOAT) FROM USERS WHERE ABOUTME !=''),2) AS posts_per_user
    FROM USERS U
    JOIN POSTS P ON U.ID = P.OWNERUSERID  
    WHERE U.ABOUTME !=''
        """
store_query("Task 26", "Considering only the users with an AboutMe, how many posts are there per user?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Not taking into account the commentcount field in the table posts, what are the Top 10 posts in terms of number of comments?
    </font></div>

In [706]:
%%sql
SELECT p.Id, COUNT(*) "Comments"
FROM posts p INNER JOIN comments c ON p.Id = c.PostId
GROUP BY p.Id
ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chatdata.db
Done.


Id,Comments
386853,66
386556,34
418910,31
395232,31
402987,27
386075,26
394118,24
402950,23
398828,23
396111,22


In [707]:
# insert the query into the queries table
sql = """
    SELECT p.Id, COUNT(*) "Comments"
    FROM posts p INNER JOIN comments c ON p.Id = c.PostId
    GROUP BY p.Id
    ORDER BY 2 DESC
    LIMIT 10
        """
store_query("Task 27", "Not taking into account the commentcount field in the table posts, what are the Top 10 posts in terms of number of comments?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
What are the Top 10 posts which have the highest cummulative (post score + comment score) score? 
    </font></div>

In [708]:
%%sql
SELECT p.Id, p.Score + SUM(c.Score)
FROM posts p INNER JOIN comments c ON p.Id = c.PostId
GROUP BY p.Id
ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chatdata.db
Done.


Id,p.Score + SUM(c.Score)
394118,306
394128,169
388578,141
398653,111
388566,101
398646,99
421677,96
420526,92
400317,72
388582,69


In [709]:
# insert the query into the queries table
sql = """
    SELECT p.Id, p.Score + SUM(c.Score)
    FROM posts p INNER JOIN comments c ON p.Id = c.PostId
    GROUP BY p.Id
    ORDER BY 2 DESC
    LIMIT 10
        """
store_query("Task 28", "What are the Top 10 posts which have the highest cummulative (post score + comment score) score?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Who are the top 10 users who comment the most?
    </font></div>

In [711]:
%%sql
SELECT u.Id "User Id", COUNT(*) "Number of Comments", u.Reputation
FROM users u INNER JOIN comments c ON u.Id = c.UserId
GROUP BY c.UserId, u.Reputation
ORDER BY 3 DESC
LIMIT 10

 * sqlite:///chatdata.db
Done.


User Id,Number of Comments,Reputation
805,1153,228662
919,3301,223056
7290,370,115531
686,349,85077
28666,158,75024
35989,470,71548
7224,424,65999
4253,255,59952
1352,422,59160
22311,467,51155


In [712]:
# insert the query into the queries table
sql = """
    SELECT u.Id "User Id", COUNT(*) "Number of Comments", u.Reputation
    FROM users u INNER JOIN comments c ON u.Id = c.UserId
    GROUP BY c.UserId, u.Reputation
    ORDER BY 3 DESC
    LIMIT 10
        """
store_query("Task 29", "Who are the top 10 users who comment the most?", sql)

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="black">
Who are the top 10 users who post the most? review
    </font></div>

In [713]:
%%sql
SELECT u.Id "User Id", COUNT(*) "Number of Post", u.Reputation
FROM users u INNER JOIN posts p ON u.Id = p.OwnerUserId
GROUP BY u.Id, u.Reputation
ORDER BY 3 DESC
LIMIT 10

 * sqlite:///chatdata.db
Done.


User Id,Number of Post,Reputation
805,230,228662
919,203,223056
7290,35,115531
686,386,85077
28666,8,75024
35989,230,71548
7224,233,65999
4253,71,59952
1352,285,59160
22311,140,51155


In [714]:
# insert the query into the queries table
sql = """
    SELECT u.Id "User Id", COUNT(*) "Number of Post", u.Reputation
    FROM users u INNER JOIN posts p ON u.Id = p.OwnerUserId
    GROUP BY u.Id, u.Reputation
    ORDER BY 3 DESC
    LIMIT 10
        """
store_query("Task 30", "Who are the top 10 users who post the most?", sql)

# 4: Check the Queries Table

Now let's tidy up and check the queries table.

First let's check it's contents:

In [715]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,SELECT * FROM queries
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table post,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 3,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 4,Count the number of rows in the comments table,SELECT Count(*) AS NumberOfRows FROM comments
Task 5,Count the number of rows in the posts table,SELECT Count(*) AS NumberOfRows FROM posts
Task 6,Count the number of rows in the users table,SELECT Count(*) AS NumberOfRows FROM users
Task 7,select 5 random rows from posts table,SELECT * FROM posts ORDER BY random() LIMIT 5
Task 8,select 5 random rows from comments table,SELECT * FROM comments ORDER BY random() LIMIT 5
Task 9,select 5 random rows from users table,SELECT * FROM users ORDER BY random() LIMIT 5


## Drop Duplicates

You likely have some duplicates. Lets drop them. 

In [716]:
# Read the queries table into pandas
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)

# Drop duplicates
queries.drop_duplicates(inplace = True) # drop duplicates

## Case Issues

Remember, SQL is case insensitive. Pandas IS case sensitive. Lets deal with this now by making all of the text uppercase. 

In [717]:
for col in queries.columns:
    queries[col] = queries[col].str.upper()
    
queries

,task,action,query
0,SINGLE TABLE QUERIES,WHICH 5 USERS HAVE VIEWED THE MOST TIMES AND W...,SELECT * FROM QUERIES
1,TASK 1,CREATE TABLE COMMENTS,"\n CREATE TABLE ""COMMENTS"" (\n ""ID"" INTE..."
2,TASK 2,CREATE TABLE POST,"\n CREATE TABLE ""POSTS"" (\n ""ID"" INTEGER..."
3,TASK 3,CREATE TABLE USERS,"\n CREATE TABLE ""USERS"" (\n ""ID"" INTEGER..."
4,TASK 4,COUNT THE NUMBER OF ROWS IN THE COMMENTS TABLE,\nSELECT COUNT(*) AS NUMBEROFROWS\n FROM CO...
5,TASK 5,COUNT THE NUMBER OF ROWS IN THE POSTS TABLE,\nSELECT COUNT(*) AS NUMBEROFROWS\n FROM PO...
6,TASK 6,COUNT THE NUMBER OF ROWS IN THE USERS TABLE,\nSELECT COUNT(*) AS NUMBEROFROWS\n FROM US...
7,TASK 7,SELECT 5 RANDOM ROWS FROM POSTS TABLE,\n SELECT * \n FROM POSTS \n ORDER BY...
8,TASK 8,SELECT 5 RANDOM ROWS FROM COMMENTS TABLE,\n SELECT * \n FROM COMMENTS \n ORDER...
9,TASK 9,SELECT 5 RANDOM ROWS FROM USERS TABLE,\n SELECT * \n FROM USERS \n ORDER BY...


In [718]:
# Write the now deduped uppercase dataframe back to sqlite and replace the table
queries.to_sql('queries', con, if_exists='replace', index=False)

31

## Use Case

Now that we have this queries table, lets give you some ideas about how you would use it.

Suppose you wanted to find all of the queries where you did a GROUP BY:

In [719]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%GROUP BY%'

 * sqlite:///chatdata.db
Done.


query
"SELECT ID ""POST ID"", SUM(VIEWCOUNT) ""TOTAL VIEWS"" FROM POSTS GROUP BY ID ORDER BY 2 DESC LIMIT 5"
"SELECT ID ""POST ID"", SUM(SCORE) ""TOTAL SCORE"" FROM POSTS GROUP BY ID ORDER BY 2 DESC LIMIT 5"
"SELECT SCORE, COUNT(SCORE) ""FREQUENT SCORES ON POSTS"" FROM POSTS GROUP BY SCORE ORDER BY 2 DESC LIMIT 5"
"SELECT COMMENTCOUNT, COUNT(COMMENTCOUNT) ""FREQUENT COMMENT COUNT"" FROM POSTS GROUP BY COMMENTCOUNT ORDER BY 2 DESC LIMIT 5"
"SELECT ID ""POST ID"", SUM(VIEWCOUNT) ""TOTAL VIEWS"", LEN_BODY ""LENGTH"" FROM POSTS GROUP BY ID ORDER BY 2 DESC LIMIT 5"
"SELECT LOCATION, COUNT(LOCATION) ""USERS"" FROM USERS GROUP BY LOCATION ORDER BY 2 DESC LIMIT 5"
"SELECT CASE CAST (STRFTIME('%W', CREATIONDATE) AS INTEGER) WHEN 0 THEN 'SUNDAY' WHEN 1 THEN 'MONDAY' WHEN 2 THEN 'TUESDAY' WHEN 3 THEN 'WEDNESDAY' WHEN 4 THEN 'THURSDAY' WHEN 5 THEN 'FRIDAY' ELSE 'SATURDAY' END AS DAY, ROUND(SUM(VIEWCOUNT)*100.0/(SELECT SUM(VIEWCOUNT) FROM POSTS),2) AS SUMVIEWCOUNTPERCENT FROM POSTS GROUP BY DAY ORDER BY 2 DESC"
"SELECT P.ID, COUNT(*) ""COMMENTS"" FROM POSTS P INNER JOIN COMMENTS C ON P.ID = C.POSTID GROUP BY P.ID ORDER BY 2 DESC LIMIT 10"
"SELECT P.ID, P.SCORE + SUM(C.SCORE) FROM POSTS P INNER JOIN COMMENTS C ON P.ID = C.POSTID GROUP BY P.ID ORDER BY 2 DESC LIMIT 10"
"SELECT U.ID ""USER ID"", COUNT(*) ""NUMBER OF COMMENTS"", U.REPUTATION FROM USERS U INNER JOIN COMMENTS C ON U.ID = C.USERID GROUP BY C.USERID, U.REPUTATION ORDER BY 3 DESC LIMIT 10"


Find the queries that have 'DISTINCT' in them. 



In [720]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%DISTINCT%'

 * sqlite:///chatdata.db
Done.


query
"SELECT COUNT(DISTINCT(LOCATION)) ""DIFFERENT LOCATIONS"" FROM USERS"


# Close SQLite



In [ ]:
con.close()